# 머신러닝 돌리기 (기본)
### history-stations
- 새로운 변수 추가, 튜닝 등 진행하지 않음

In [ ]:
!pip install pymongo

#### 라이브러리 준비

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from prophet import Prophet
import holidays

from datetime import datetime, timedelta
now = datetime.now()

import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

#### 방문자수 데이터 데이터 프레임 변환

In [ ]:
from pymongo import MongoClient

# 데이터가 저장된 MongoDB의 주소
client = MongoClient('주소 입력')

# db를 저장하기
db = client.crawling

collection = db['collection 입력']

# collection에 저장된 데이터를 데이터프레임으로 변환 및 저장
rows = collection.find()
history_stations = []
for row in rows:
    history_stations.append(row)

history_stations = pd.DataFrame(history_stations)

#### 머신러닝 성능 테스트 ⭐
(초기 0.65)
- 평가 기준 r2(결정계수)
-  1. RandomForestRegressor 👍👍
-  결과: 0.7486
-  2. LinearRegression
-  결과: 0.2363
-  3. LGBMRegressor
-  결과: 0.7023
-  4. xgboost
-  결과: 0.6801

#### 모든 충전소 정보를 하나로 합치기 => df_all

In [ ]:
# history_stations의 각 항목을 리스트로 저장
dfs = []

# 첫 번째 항목 처리
for i in range(len(history_stations)):
    df_temp = pd.DataFrame(history_stations['history'][i])  # history 항목을 DataFrame으로 변환
    df_temp['_id'] = history_stations['_id'][i]  # _id 값을 추가
    dfs.append(df_temp)  # 각 DataFrame을 리스트에 저장

# 모든 DataFrame을 한 번에 concat
df_all = pd.concat(dfs, ignore_index=True)

# 결과 확인
df_all.tail()

#### 임의의 날짜 변수 추가

In [ ]:
# 변수 생성 (주말, 월, 일, 시간, 분) #########################
df_all['weekday'] = df_all['time'].dt.weekday
df_all['month'] = df_all['time'].dt.month
df_all['day'] = df_all['time'].dt.day
df_all['hour'] = df_all['time'].dt.hour
df_all['minute'] = df_all['time'].dt.minute

# 변수 생성 (공휴일)
kr_holidays = holidays.KR()
df_all['holiday'] = df_all.time.apply(lambda x: 1 if x in kr_holidays else 0)

# ml 실행을 위해 날짜를 index로 설정하기
df_all.set_index(keys='time', inplace=True)

In [ ]:
len(df_all['_id'].value_counts())

#### 충전소 _id 값을 인코딩하기
- id 값이 고유한 카테고리인데, Label Encoding은 숫자 순서가 크기를 암묵적으로 부여하기 때문에 모델이 그 사이의 숫자 차이를 학습할 수 있다.
- 예를 들어, id 0과 id 1이 다를 뿐인데, 모델은 이 차이를 수치적으로 다르게 취급할 수 있다.
- id 값을 0, 1, 2...와 같은 연속적인 정수로 변환하면, 모델이 이 값들 간의 순서 관계나 크기 차이를 학습할 수 있게 되는데, 이는 실제로 의미가 없을 수 있다.
- One Hot Encoding은 충전소의 수가 2471 + α이기 때문에 변수가 너무 많이 생겨 사용할 수 없다.
- 그러므고 가장 유용하다고 판단되는 target_encoding을 활용한다.

#### target encoding 적용

In [ ]:
# 1. 각 '_id'에 대해 target 값(visitNum)의 평균을 구하기
target_means = df_all.groupby('_id')['visitNum'].mean()

# 2. '_id' 컬럼을 해당 평균 값으로 대체하기
df_all['id_encoded'] = df_all['_id'].map(target_means)

# _id 컬럼 삭제하기
df_all = df_all.drop('_id', axis=1)

In [ ]:
# count 변수 저장하기
count = df_all.iloc[0, 0]

# target(예측할 열) 설정하기
target = 'visitNum'

# x, y 값 설정하기
x = df_all.drop(target, axis=1)
y = df_all[target]


# 데이터가 10개 이상일 경우, 기존처럼 훈련/테스트 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=0)

model = RandomForestRegressor(random_state=0, n_jobs=-1)

model.fit(x_train, y_train) # 모델 학습

y_pred = model.predict(x_test) # 모델 예측
y_pred = np.round(y_pred)

print(r2_score(y_test, y_pred))  # 모델 정확도 출력
acc = model.score(x_test, y_test)

# 시각화를 위함 - df 형식으로 변환
y_pred = pd.DataFrame(y_pred)
y_test = pd.DataFrame(y_test)

# 하나의 df로 합치기
y_test.reset_index(inplace=True)
df = pd.concat([y_test, y_pred], axis=1)
df.columns = ['time', 'y_test', 'y_pred']

In [ ]:
# 'y_test'와 'y_pred' 열을 비교하여 정확도 평가

# R² (R-squared) 평가
r2 = r2_score(df['y_test'], df['y_pred'])
print(f"R-squared: {r2:.4f}")

# MSE (Mean Squared Error) 평가
mse = mean_squared_error(df['y_test'], df['y_pred'])
print(f"Mean Squared Error: {mse:.4f}")

# RMSE (Root Mean Squared Error) 평가
rmse = mse ** 0.5
print(f"Root Mean Squared Error: {rmse:.4f}")

In [ ]:
# 시각화
plt.figure(figsize=(8,6))
sns.lineplot(x='time' , y='y_test', data=df)
sns.lineplot(x='time' , y='y_pred', data=df)
plt.xticks(rotation=50)
plt.show()

In [ ]:
# 일정 간격으로 데이터 샘플링 (예: 2471개마다 하나씩 선택)
df_resampled = df.iloc[::2471, :]

plt.figure(figsize=(8,6))
sns.lineplot(x='time', y='y_test', data=df_resampled, label='Actual')
sns.lineplot(x='time', y='y_pred', data=df_resampled, label='Predicted')
plt.xticks(rotation=50)
plt.show()

#### 실제 데이터로 예측 후 MongoDB에 반영하기 🌟

In [ ]:
def run_ml(i):
    # 시간 단위별 예측 df 생성
    pre_df = pd.date_range(now.date()+ timedelta(days=1) , periods=24 , freq="30min") # 30분 단위로 예측 df 만들기

    pre_df = pd.DataFrame(pre_df) # 데이터 프레임 형태로 변환
    pre_df.columns=['time'] # 열 이름 변경

    id = history_stations['_id'][i]

    pre_df['count'] = count

    # 변수 추가하기
    pre_df['time'] = pd.to_datetime(pre_df['time'])
    pre_df['weekday'] = pre_df['time'].dt.weekday
    pre_df['month'] = pre_df['time'].dt.month
    pre_df['day'] = pre_df['time'].dt.day
    pre_df['hour'] = pre_df['time'].dt.hour
    pre_df['minute'] = pre_df['time'].dt.minute

    kr_holidays = holidays.KR()
    pre_df['holiday'] = pre_df.time.apply(lambda x: 1 if x in kr_holidays else 0)

    pre_df.set_index(keys='time', inplace=True)

    #### target encoding을 사용하려고 하는데 각각 target을 i값 별로 다르기 때문에
    pre_df['id_encoded'] = target_means.iloc[i]


    pre_predict = model.predict(pre_df)
    pre_predict= np.round(pre_predict)

    pre_predict = pre_predict.tolist()
    collection = db['demand-info']

    statId = history_stations['_id'][i]
    # 해당 statId를 가진 문서 조회

    # print(id, statId)
    # print(pre_predict)
    existing_doc = collection.find_one({"statId": statId })

    # 문서가 존재하지 않으면 새로운 문서를 추가하고 업데이트
    if existing_doc is None:
        new_doc = { "statId": statId, "demandInfo": { "viewNum": 0, "departsIn30m": [], "hourlyVisitNum": [] } }
        result = collection.insert_one(new_doc)
        print("Added new document")
        #time.sleep(0.1)

    x = collection.update_one(
        {"statId":statId},
        {"$set" : {
            'demandInfo.hourlyVisitNum' : pre_predict
        }
        })

In [ ]:
for i in range(len(history_stations)):
    run_ml(i)

#### 하나의 데이터프레임으로 합친 뒤 성능
- RandomForestRegressor : 0.9388
- GridSearchCV 교차 검증을 통해 최적의 하이퍼파라미터 출력한 RandomForestRegressor: (미적용)

#### 계획: k-means로 유사한 특징을 가진 충전소들별로 모델 따로 학습 및 적용하기